In [ ]:
!pip install --q --upgrade pip
!pip install --q --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]
!pip install --q flash-attn --no-build-isolation
!pip install gradio==3.50 -q
!pip install -q git+https://github.com/suno-ai/bark.git
!pip install git+https://github.com/huggingface/transformers -q peft  accelerate bitsandbytes safetensors sentencepiece
!pip install -i https://pypi.org/simple/ bitsandbytes -q
!pip install accelerate -q
!pip install -q ffmpeg-python

In [2]:

import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Directory paths for saving the models
save_dir = "/content/drive/My Drive/saved_models"
text_to_speech_dir = f"{save_dir}/text_to_speech"
speech_to_text_dir = f"{save_dir}/speech_to_text"
mistral_dir = f"{save_dir}/mistral"



Mounted at /content/drive


# Wishper

In [ ]:
!nvidia-smi

Thu Apr 11 09:04:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
torch_dtype

torch.float16

In [ ]:
model_id = "openai/whisper-large-v3"

In [ ]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
pipe

In [ ]:
audio = "/content/Speech on Importance of Reading in English  Benefits of Reading.mp3"

In [ ]:
%%timeit
result = pipe(audio) # v2 gives better results if you don't provide a language.

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


6.73 s ± 778 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
result = pipe(audio)

In [ ]:
result['text']

' Importance of Reading Speech for Higher Secondary Students Good morning respected teachers and my dear friends. Today I am standing before you to express my views about importance of reading. Reading is a habit that can make us all a better person. Reading gives us knowledge and inspiration. It makes us smart. There are numerous examples of great people who emphasize on reading for a good and successful life. Reading exposes our mind to a great treasure of ideas and thoughts. People share their experiences in life and work through books. Reading such books can be of great help. Reading novels, biographies, stories, poems, etc. entertain us. It helps to relax our mind. We try to visualize whatever we read. This definitely is a healthy exercise for our mind. Reading improves our concentration power. It improves our emotional health too. Reading books can make us feel happy, sad, loved and so on. Ultimately a good book helps us grow emotionally. When we read books we remember a lot of t

In [ ]:
result['chunks']

[{'timestamp': (1.0, 5.0),
  'text': ' Importance of Reading Speech for Higher Secondary Students'},
 {'timestamp': (6.0, 10.0),
  'text': ' Good morning respected teachers and my dear friends.'},
 {'timestamp': (10.0, 17.74),
  'text': ' Today I am standing before you to express my views about importance of reading.'},
 {'timestamp': (17.74, 21.94),
  'text': ' Reading is a habit that can make us all a better person.'},
 {'timestamp': (21.94, 26.42),
  'text': ' Reading gives us knowledge and inspiration. It makes us smart. There'},
 {'timestamp': (26.42, 37.46),
  'text': ' are numerous examples of great people who emphasize on reading for a good and successful life. Reading exposes our mind to a great treasure'},
 {'timestamp': (37.46, 46.6),
  'text': ' of ideas and thoughts. People share their experiences in life and work through books. Reading such books can be of great help.'},
 {'timestamp': (47.9, 57.0),
  'text': ' Reading novels, biographies, stories, poems, etc. entertain u

# Distilled wishper

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from optimum.bettertransformer import BetterTransformer

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
model_id = "distil-whisper/distil-medium.en"
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
# Load speech-to-text model
speech_to_text_model = AutoModelForSpeechSeq2Seq.from_pretrained("/content/drive/My Drive/saved_models/speech_to_text").to(device)

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_to_text_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
%%timeit
result = pipe("/content/audio_recording.wav")

In [ ]:
result = pipe("/content/audio_recording.wav")

# text to speech

In [ ]:
from transformers import AutoModel, AutoProcessor
from IPython.display import Audio, display, HTML
import torch
import base64 # Import the base64 module
import io
from scipy.io.wavfile import write as wav_write

text_to_speech_processor = AutoProcessor.from_pretrained("suno/bark-small")
voice_preset = "v2/en_speaker_6"

# Load text-to-speech model
text_to_speech_model = AutoModel.from_pretrained("/content/drive/My Drive/saved_models/text_to_speech").to("cuda")

def generate_audio_from_text(text, voice_preset="v2/en_speaker_6"):
    # Prepare the inputs for the model
    inputs = text_to_speech_processor(
        text=[text],
        return_tensors="pt",
        voice_preset=voice_preset
    )
    inputs = inputs.to("cuda")

    # Generate the speech
    speech_values = text_to_speech_model.generate(**inputs, do_sample=True)
    sampling_rate = text_to_speech_model.generation_config.sample_rate

    # Convert the generated speech to audio
    audio_data = speech_values.cpu().numpy().squeeze()
    audio_bytes = io.BytesIO()
    wav_write(audio_bytes, sampling_rate, audio_data)
    audio_bytes.seek(0)
    audio_url = f"data:audio/wav;base64,{base64.b64encode(audio_bytes.read()).decode()}"

    # Create an HTML audio element and set it to play automatically
    audio_html = f"""
    <audio controls autoplay>
        <source src="{audio_url}" type="audio/wav">
        Your browser does not support the audio element.
    </audio>
    """

    # Display the audio element
    display(HTML(audio_html))


In [ ]:

# Example usage
text = "Hello, my name is Tushar. I am making a text-to-speech model."
generate_audio_from_text(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


# INterviw questiona

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = 'filipealmeida/Mistral-7B-Instruct-v0.1-sharded'

def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

    return model

def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

def perform_inference(text: str, model_name: str, max_new_tokens: int = 200):
    """
    Perform inference using the specified text and model.

    :param text: Input text for inference.
    :param model_name: Name or path of the model.
    :param max_new_tokens: Maximum number of tokens to be generated.
    :return: Decoded output from the model.
    """
    device = next(model_name.parameters()).device
    # Tokenize the input text
    encoded = tokenizer(text, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = {k: v.to(device) for k, v in encoded.items()}
    # Perform inference
    generated_ids = model_name.generate(**model_input, max_new_tokens=max_new_tokens, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    # Remove the instruction part from the output
    instruction_part = text.strip()
    output_without_instruction = decoded[0].replace(instruction_part, '').strip().replace('</s>', '')

    return output_without_instruction

In [ ]:
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mistral_model = AutoModelForCausalLM.from_pretrained("/content/drive/My Drive/saved_models/mistral")
tokenizer = initialize_tokenizer(model_name)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
# Example usage:
text_input = "[INST] How AI will replace Engineers [/INST]"
generated_output = perform_inference(text_input, mistral_model)
print(generated_output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


AI has the potential to of the field that many jobs in engineering, including those that are traditionally done by humans. However, it is also true that AI will create new jobs and a new role for engineers in the field.

One way AI may replace engineers is through the use of automation and robots that can perform tasks that were previously done by people. For example, autonomous vehicles could eliminate the need for human drivers, while robots could perform tasks such as assembly and manufacturing in factories.

Another way AI may replace engineers is through the use of machine learning algorithms, which can analyze data and make predictions or decisions on their own. This could lead to the automation of tasks such as pattern recognition, data analysis and prediction, quality control, and customer service.

However, it's important to note that AI is not a replacement for human engineers, but rather a tool that can help them to design and build more efficient and effective systems. Engi

In [ ]:
# Example usage:
text_input = "[INST] Generate an introductory interview question that assesses the candidate's background, experience, and motivations for applying to this position of blockchain engineer.[/INST]"
generated_output = perform_inference(text_input, mistral_model)
print(generated_output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can you tell us about your background and experience in blockchain technology, and what motivated you to apply for the position of blockchain engineer at our company?


In [ ]:
# Example usage:
text_input = "[INST] Based on the candidate's response to the  question,which is 'My background in computer science and engineering, along with experience in blockchain development, equips me well for a role as a blockchain engineer. My motivation lies in leveraging my skills to contribute to innovative projects and collaborate with a dynamic team.' generate a follow-up question that dives deeper into their relevant experience or skills.[/INST]"
generated_output = perform_inference(text_input, model_name)
print(generated_output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Can you describe a specific blockchain project you have worked on and the challenges you faced while developing it?


# loop

In [4]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import time

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  resolve(base64data.toString())
});

}
});

</script>
"""

def get_audio():
    time.sleep(3)
    print("Press Enter to start recording")
    input()
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    process = (
        ffmpeg.input('pipe:0')
        .output('pipe:1', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    # Save the audio file
    audio_file_path = "/content/audio_recording.wav"
    wav_write(audio_file_path, sr, audio)

    return audio_file_path


In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,AutoModel
from IPython.display import Audio, display, HTML
import base64 # Import the base64 module
import io
from scipy.io.wavfile import write as wav_write

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

'''speech to text'''
model_id = "distil-whisper/distil-medium.en"
# Load speech-to-text model
speech_to_text_model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id).to(device)
speech_to_text_processor = AutoProcessor.from_pretrained(model_id)

speech_to_text_pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_to_text_model,
    tokenizer=speech_to_text_processor.tokenizer,
    feature_extractor=speech_to_text_processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)



'''text to speech'''
text_to_speech_processor = AutoProcessor.from_pretrained("suno/bark-small")
voice_preset = "v2/en_speaker_6"

# Load text-to-speech model
text_to_speech_model = AutoModel.from_pretrained("suno/bark-small").to("cuda")

'''mistral'''
model_name = 'filipealmeida/Mistral-7B-Instruct-v0.1-sharded'
mistral_model = AutoModelForCausalLM.from_pretrained(model_name)
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer
mistral_tokenizer = initialize_tokenizer(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [22]:
def perform_inference(text: str, model_name: str, max_new_tokens: int = 50,output_file="question.txt"):
    """
    Perform inference using the specified text and model.

    :param text: Input text for inference.
    :param model_name: Name or path of the model.
    :param max_new_tokens: Maximum number of tokens to be generated.
    :return: Decoded output from the model.
    """
    device = next(model_name.parameters()).device
    # Tokenize the input text
    encoded = mistral_tokenizer(text, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = {k: v.to(device) for k, v in encoded.items()}
    # Perform inference
    generated_ids = model_name.generate(**model_input, max_new_tokens=max_new_tokens, do_sample=True)
    decoded = mistral_tokenizer.batch_decode(generated_ids)
    # Remove the instruction part from the output
    instruction_part = text.strip()
    output_without_instruction = decoded[0].replace(instruction_part, '').strip().replace('</s>', '')

    if output_file:
        with open(output_file, 'a') as f:  # Open file in append mode
            f.write("Previous Question:" + output_without_instruction + "\n")  # Append new output line

    return output_without_instruction


def analyze_response(path, question_file="question.txt"):
    # Read the candidate's response
    answer = speech_to_text_pipe(path)['text']

    # Read the previous questions from the question file
    with open(question_file, 'r') as f:
        lines = f.readlines()
        previous_questions = []
        for line in reversed(lines):
            # Check if the line starts with a number
            if line.strip() and line.strip()[0].isdigit():
                previous_questions.insert(0, line.strip())  # Insert at the beginning to preserve order
            else:
                # If the line doesn't start with a number, this means it's the last question
                last_question = line.strip()
                break
    # Construct the prompt with the candidate's response and the previous questions
    next_text = f"""
    [INST]
    Interview Question Prompt:
    Previous Questions: {' '.join(previous_questions)}
    Based on the candidate's response to the previous question, which was: '{last_question}', please formulate the next question to delve deeper into the candidate's skills and experiences.

    When crafting your question(s):
    - Be specific and focused on a particular aspect of the candidate's background or experience.
    - Encourage the candidate to provide detailed examples or scenarios.
    - Consider the relevance of the previous questions and the candidate's response to the role and its requirements.

    Remember to always consider the candidate's response when formulating the next question.
    [/INST]
    """

    # Generate the next question using the LLM model
    next_question = perform_inference(next_text, mistral_model)

    return next_question


def generate_audio_from_text(text, voice_preset="v2/en_speaker_6"):
    # Prepare the inputs for the model
    inputs = text_to_speech_processor(
        text=[text],
        return_tensors="pt",
        voice_preset=voice_preset
    )
    inputs = inputs.to("cuda")

    # Generate the speech
    speech_values = text_to_speech_model.generate(**inputs, do_sample=True)
    sampling_rate = text_to_speech_model.generation_config.sample_rate

    # Convert the generated speech to audio
    audio_data = speech_values.cpu().numpy().squeeze()
    audio_bytes = io.BytesIO()
    wav_write(audio_bytes, sampling_rate, audio_data)
    audio_bytes.seek(0)
    audio_url = f"data:audio/wav;base64,{base64.b64encode(audio_bytes.read()).decode()}"

    # Create an HTML audio element and set it to play automatically
    audio_html = f"""
    <audio controls autoplay onended="audioEndedCallback()">
        <source src="{audio_url}" type="audio/wav">
        Your browser does not support the audio element.
    </audio>
    """

    # Define the JavaScript callback function to signal the end of audio playback
    js_callback = """
    <script>
    function audioEndedCallback() {
        // Remove the audio element from the output
        const audioElement = document.querySelector('audio');
        audioElement.parentNode.removeChild(audioElement);
    }
    </script>
    """

    # Display the audio element and JavaScript callback
    display(HTML(audio_html + js_callback))


In [ ]:
post_name = input("Give the post name you want to generate questions about")
# Prompt the LLM to generate the first interview question
print("Generating the first interview question...")
question = f"[INST]Generate an introductory interview question that assesses the candidate's background, experience, and motivations for applying to this position of {post_name}.[\INST]"
first_question = perform_inference(question, mistral_model)

# Use the text-to-speech model to present the generated question to the candidate
print("Question: ", first_question)
generate_audio_from_text(first_question)
while True:
  candidate_response = get_audio()
  # Analyze the candidate's response and determine the next interview question
  print("Analyzing the candidate's response...")
  next_question = analyze_response(candidate_response)

  # Repeat the process of presenting the question, capturing the response, and storing it for further analysis
  print("Next Question: ", next_question)
  generate_audio_from_text(next_question)


In [ ]:
import gradio as gr
from IPython.display import display, Audio
from base64 import b64decode
import io
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, AutoModelForCausalLM, AutoTokenizer
import time
from scipy.io.wavfile import write as wav_write
import ffmpeg

# Load speech-to-text model
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model_id = "distil-whisper/distil-medium.en"
# speech_to_text_model = AutoModelForSpeechSeq2Seq.from_pretrained("/content/drive/My Drive/saved_models/speech_to_text").to(device)
# speech_to_text_processor = AutoProcessor.from_pretrained(model_id)

def speech_to_text(audio):
    binary = b64decode(audio.split(',')[1])

    process = (
        ffmpeg.input('pipe:0')
        .output('pipe:1', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    return {"text": speech_to_text_pipe(audio)['text']}

# Load text-to-speech model
# text_to_speech_processor = AutoProcessor.from_pretrained("suno/bark-small")
# voice_preset = "v2/en_speaker_6"
# text_to_speech_model = AutoModel.from_pretrained("/content/drive/My Drive/saved_models/text_to_speech").to("cuda")

def text_to_speech(text):
    inputs = text_to_speech_processor(
        text=[text],
        return_tensors="pt",
        voice_preset=voice_preset
    )
    inputs = inputs.to("cuda")

    speech_values = text_to_speech_model.generate(**inputs, do_sample=True)
    sampling_rate = text_to_speech_model.generation_config.sample_rate

    audio_data = speech_values.cpu().numpy().squeeze()
    audio_bytes = io.BytesIO()
    wav_write(audio_bytes, sampling_rate, audio_data)
    audio_bytes.seek(0)
    audio_url = f"data:audio/wav;base64,{base64.b64encode(audio_bytes.read()).decode()}"

    return Audio(audio_data, rate=sampling_rate)

# Load Mistral model for generating questions
# model_name = 'filipealmeida/Mistral-7B-Instruct-v0.1-sharded'
# mistral_model = AutoModelForCausalLM.from_pretrained("/content/drive/My Drive/saved_models/mistral")
# mistral_tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate_interview_question(post_name):
    question = f"[INST]Generate an introductory interview question that assesses the candidate's background, experience, and motivations for applying to this position of {post_name}.[\INST]"
    return perform_inference(question, mistral_model)

def perform_inference(text: str, model_name: str, max_new_tokens: int = 50,output_file="question.txt"):
    device = next(model_name.parameters()).device
    encoded = mistral_tokenizer(text, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = {k: v.to(device) for k, v in encoded.items()}
    generated_ids = model_name.generate(**model_input, max_new_tokens=max_new_tokens, do_sample=True)
    decoded = mistral_tokenizer.batch_decode(generated_ids)
    instruction_part = text.strip()
    output_without_instruction = decoded[0].replace(instruction_part, '').strip().replace(' ', '')

    if output_file:
        with open(output_file, 'a') as f:
            f.write("Previous Question:" + output_without_instruction + "\n")

    return output_without_instruction

def interview_loop(post_name):
    question = generate_interview_question(post_name)
    audio_question = text_to_speech(question)
    display(audio_question)
    time.sleep(3)
    # Use Microphone input component for recording audio
    response_audio = gr.inputs.Microphone()
    # Assuming you have a function to handle the audio input and generate a response
    response_text = handle_audio_input(response_audio)
    next_question = analyze_response(response_text)
    audio_next_question = text_to_speech(next_question)
    display(audio_next_question)

# Launch the Gradio interface
iface_interview_loop = gr.Interface(interview_loop, inputs="text", outputs=None, title="Interview Question Generator")
iface_interview_loop.launch(debug=True)
